In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
from typing import List
import instructor
from pydantic import BaseModel

class Category(BaseModel):
    primary: str
    secondary: str

class ResultDetail(BaseModel):
    exist: str
    category: Category | None = None
    freshness: str
    description: str

class ResultList(BaseModel):
    result: List[ResultDetail] = []
    description: str


client = instructor.patch(openai.OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_API_BASE"),
), mode=instructor.function_calls.Mode.MD_JSON)

gpt_prompt1='''任务简介：
- 识别每张图片中的主要物品，判断其分类并评估新旧程度。

分类标准：
- 服装：大衣、皮衣、半袖、裤子、裙子、内衣裤、秋衣类、毛衣类、工装、校服。
- 家电：空气净化器、厨房家电、家居家电。
- 乐器：电子乐器、琴类。
- 玩具：小件（手办）、大件、毛绒玩具。
- 图书：儿童绘本、课外书、小说、套装书籍、课本。
- 手机：智能手机、功能机。
- 笔记本电脑：品牌电脑、非品牌电脑。

对于没有给出分类的物品，需要你自行判断一级分类（primary）和二级分类（secondary），此时exist设置为"否"。
'''

def describe_multiple(images, prompt=gpt_prompt1):
    response = client.chat.completions.create(
        model="gpt-4-vision-preview",
        response_model=ResultList,
        messages=[{
            "role": "user",
            "content": [{"type": "text", "text": prompt},] + 
            [{"type": "image_url", "image_url": {"url": image, "detail": "low",}} for image in images]
        }],
        max_tokens=3000,
        temperature=0,
    )
    return response

In [2]:
urls = [f"https://milo-test.oss-cn-zhangjiakou.aliyuncs.com/hdd/batch1/image{i+1:03}.png" for i in range(241)]

In [3]:
from IPython.display import Image, display, HTML

In [15]:
%%time
ret = describe_multiple(urls[:1])

CPU times: user 15 ms, sys: 2.26 ms, total: 17.2 ms
Wall time: 1min 9s


In [16]:
table = "<table><tr><th>Text</th><th>Image</th></tr>"
for i in range(1):
    table += f"<tr><td><img src='{urls[i]}' width='100'></td><td>{ret.result[i]}</td></tr>"    

In [17]:
display(HTML(table))

Text,Image
,"exist='否' category=Category(primary='运动鞋', secondary='') freshness='新' description='白色运动鞋，有跑步绳和三个带子。'"


In [19]:
display(Image(url=urls[:1][0]))